In [10]:
from commonfunctions import *
import cv2 as cv

In [11]:
# Path to the dataset
DATASET_PATH = "./English/Img"  # Chars74K dataset

# Constants
IMAGE_SIZE = (64, 64)  # Resizes images to 64x64
NUM_CLASSES = 36  # 26 letters + 10 digits

def load_chars74k(dataset_path=DATASET_PATH):
    """Load the Chars74K dataset and preprocess images from both GoodImg and BadImg."""
    images = []
    labels = []

    # Define subfolders for GoodImg and BadImg
    folders = ['GoodImg/Bmp', 'BadImag/Bmp']

    for folder in folders:
        folder_path = os.path.join(dataset_path, folder)

        # Iterate over all character folders in GoodImg and BadImg
        for label, char_folder in enumerate(sorted(os.listdir(folder_path))):
            char_folder_path = os.path.join(folder_path, char_folder)
            if not os.path.isdir(char_folder_path):
                continue

            # Iterate over all images in the character folder
            for img_name in os.listdir(char_folder_path):
                img_path = os.path.join(char_folder_path, img_name)
                img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)

                # Resize and normalize the image
                img = cv.resize(img, IMAGE_SIZE)
                img = img / 255.0
                images.append(img)
                labels.append(label)

    # Convert to numpy arrays
    images = np.array(images).reshape(-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 1)
    labels = np.array(labels)

    return images, labels


def build_cnn_model(input_shape=(64, 64, 1), num_classes=NUM_CLASSES):
    """Build and compile a CNN model."""
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation="relu"),
        Dropout(0.5),
        Dense(num_classes, activation="softmax"),
    ])

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model


def runOCR():
    # Load and preprocess the dataset
    print("Loading dataset...")
    images, labels = load_chars74k()
    print(f"Loaded {len(images)} images and {len(labels)} labels.")

    # One-hot encode labels
    labels = to_categorical(labels, NUM_CLASSES)

    # Split into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Build the CNN model
    print("Building model...")
    model = build_cnn_model()

    # Train the model
    print("Training model...")
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

    # Evaluate the model
    print("Evaluating model...")
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save the model
    model.save("chars74k_model.h5")
    print("Model saved as chars74k_model.h5")


runOCR()

Loading dataset...
Loaded 9153 images and 9153 labels.
Building model...


/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training model...
Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 30s 126ms/step - accuracy: 0.0858 - loss: 3.3592 - val_accuracy: 0.2840 - val_loss: 2.8000
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 29s 126ms/step - accuracy: 0.3654 - loss: 2.4462 - val_accuracy: 0.6073 - val_loss: 1.3752
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 29s 126ms/step - accuracy: 0.6009 - loss: 1.4381 - val_accuracy: 0.7389 - val_loss: 0.9929
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 35s 153ms/step - accuracy: 0.7257 - loss: 0.9741 - val_accuracy: 0.7673 - val_loss: 0.8572
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 31s 133ms/step - accuracy: 0.7682 - loss: 0.8164 - val_accuracy: 0.7854 - val_loss: 0.7744
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 28s 123ms/step - accuracy: 0.7925 - loss: 0.6974 - val_accuracy: 0.7886 - val_loss: 0.7241
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 31s 134ms/step - accuracy: 0.8176 - loss: 0.6141 - val_accuracy: 0.8007 - val_loss: 0.7201
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 36s 157ms/step - accuracy

Test Accuracy: 81.87%
Model saved as chars74k_model.h5
